<a href="https://colab.research.google.com/github/rchejfec/IRPP-oasis-llm-automation-ratings-guide/blob/main/IRPP_oasis_llm_automation_ratings_guide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Introduction

This notebook walks through the process of using a Large Language Model (LLM) via the DSPy framework to assess the automatability of skills and work activities from ESDC's OaSIS, based on Oschinski & Walia (2025).

The easiest way to get started, especially if you're new to Python or LLMs, is to use Google Colab. It's free and relatively easy to use.
1.  Ensure you are logged into a Google account.
2.  Open this notebook file in Colab.
3.  The necessary data files (`OaSIS_Guide_2023.csv` and `prompt_templates.json`) will be loaded in the first codeblock into a `data/` subdirectory within the same project structure.

If you are not using Colab or a similar cloud-based developer environment, you'll need to manage your Python environment, dependencies (like DSPy, pandas, etc.), and API keys locally. While not overly complicated, this can be more involved for those new to it.

**Notebook Steps:**
1.  **Data Preparation**: Load and preprocess skills/work activities from OaSIS and prompt templates.
2.  **LLM Configuration**: Set up the chosen LLM (e.g., Gemini, OpenAI model) with DSPy.
3.  **Prompting and Rating**: Iterate through items and prompts, sending requests to the LLM.
4.  **Results Processing**: Aggregate and analyze the LLM's ratings.


In [2]:
# Clone the entire repository from GitHub to access the data files
!git clone https://github.com/rchejfec/IRPP-oasis-llm-automation-ratings-guide.git

# Navigate into the cloned repository's directory
%cd IRPP-oasis-llm-automation-ratings-guide

Cloning into 'IRPP-oasis-llm-automation-ratings-guide'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 82 (delta 35), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (82/82), 63.95 KiB | 528.00 KiB/s, done.
Resolving deltas: 100% (35/35), done.
/content/IRPP-oasis-llm-automation-ratings-guide


In [3]:
!pip install dspy-ai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.0/259.0 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 17.5 MB/s eta 0:00:00


In [11]:
# If you are not running this in Google Colab, you'll need to install these libraries
# in your local Python environment. dspy-ai includes most dependencies.

import dspy
import pandas as pd
import numpy as np
import os
import re
import time
import json
from datetime import datetime

### Step 1: Data Preparation

**1.1. OaSIS Framework Data**

We'll first get a list of all unique skills and work activities from ESDC's OaSIS framework. The necessary file (`OaSIS_Guide_2023.csv`) is included in the `Data/` subdirectory of this repository.
The code below loads this file, extracts the relevant skills and work activities, and cleans up their descriptions.
If you wanted to use a different data source (e.g., O*Net), you would need to adapt the loading and processing steps and ensure the final list of items to rate is a Python list of dictionaries with 'Name' and 'Description' keys in each item.

In [12]:
try:
  # OaSIS Guide:
  guide_df = pd.read_csv("Data/OaSIS_Guide_2023.csv")

  # Extract skills & work activities
  items_to_rate_df = guide_df[(guide_df['Structure type'] == "Descriptor") &
                          (guide_df['Category'].isin(["Skills", "Work Activities"]))]

  # Convert to dict for easier wrangling
  items_to_rate = items_to_rate_df[['Name', 'Description']].to_dict(orient="records")

  # Remove unecessary information that might interfere with prompts.
  pattern_to_remove = r"\s*This descriptor is measured by .*? level on a scale of \d+-\d+\.?\s*$"

  for item in items_to_rate:
    original = item.get("Description")
    cleaned = re.sub(pattern_to_remove, "", original, flags=re.IGNORECASE).strip()
    item["Description"] = cleaned

  print(f"Extracted list of dictionaries with {len(items_to_rate)} items with {list(items_to_rate[0].keys())} as keys.")

except Exception as e:
  print(f"Failed to make a list of dictionaries from input table. Ensure you have uploaded the file and that file names are aligned.\
  \nError message: {e}")

Extracted list of dictionaries with 99 items with ['Name', 'Description'] as keys.


**1.2 Prompt Templates**

Next, we'll import a list of different prompt phrasings to use when querying the LLM. These phrasings come from Appendix C of the study, and are stored in `data/prompt_templates.json`. This notebook uses 10 distinct phrasings, omitting versions 7 and 9 from the original study for easier parameter handling across different models.

The prompts are designed to incorporate the name and (sometimes) the description of the skills and work activities using placeholders (e.g., `{item_name}`, `{item_descriptor}`). This helps the LLM parse the question, as models vary in their ability to process longer, formally structured requests.

You can edit the `prompt_templates.json` file using a text editor to remove, modify, or add new phrasings. For example, while only some of the original study's prompts included a description of the skill/activity, you could ensure all of them do by adding the `{item_descriptor}` placeholder where appropriate.

In [9]:
try:
  with open("Data/prompt_templates.json", "r") as f:
    prompts = json.load(f)
    print(f"Imported {len(prompts)} prompts.")
except Exception as e:
  print(f"Failed to import list of prompts. Ensure you have uploaded the file and that file names are aligned.\
  \nError message: {e}")

Imported 10 prompts.


### Step 2 - Configure LLM and DSPy

The next step is to initialize and configure the LLM and define how we'll interact with it using the DSPy framework.

This notebook is configured by default to use Google's `gemini/gemma-3-1b-it` model. **This specific model was chosen for demonstration and testing purposes in this notebook, primarily because it is relatively fast, available with free tiers, and comes with generous API rate limits, making it suitable for running the ~1,000 requests generated by this script without incurring significant costs.** However, DSPy's flexibility allows you to easily use other models like OpenAI's GPT series, Llama models via Groq, or more powerful Gemini models from Google. You'll typically need to find the correct model identifier string for your chosen provider (e.g., `openai/gpt-3.5-turbo`, `google/gemini-1.5-pro-latest`). For more robust research, consider a larger, more capable model, keeping in mind potential costs and rate limits.

**Crucially, you must handle your API key securely:**
* Do not share or accidentally publish your API key.
* The code below is set up to use Google Colab's "Secrets" feature (accessed via the 🔑 key icon on the left navigation bar). Add your API key there, give it a name (the code defaults to looking for `GOOGLE_API_KEY`), and ensure notebook access is enabled.
* Remember to update the `API_KEY_NAME` variable in the code if your secret has a different name.
* If running locally, manage your API key using environment variables or other secure methods. For guidance on local key management, you can consult resources like [this article on managing API keys in Python projects](https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety).

In [15]:
# Define the name and version of the LLM (Defaults to Gemma)
LLM_PROVIDER_MODEL_STRING = "gemini/gemma-3-1b-it"

# Initialize variables
API_KEY = None
llm = None
API_KEY_NAME = "GOOGLE_API_KEY"   # update with your own name if different

# Load API Key
# If running in Google Colab, set your API key in the "Secrets" tab.
try:
  from google.colab import userdata
  API_KEY = userdata.get(API_KEY_NAME)
  if API_KEY:
      print(f"{API_KEY_NAME} loaded from Colab secrets.")
except ImportError:
  print("Could not import Colab userdata")
  pass

# If not using Google Colab, implement your key loading here.
# A common method is using environment variables:
# import os
# API_KEY = os.getenv("YOUR_API_KEY_ENVIRONMENT_VARIABLE_NAME")

# Set model parameters
if API_KEY:
  try:
      llm = dspy.LM(
          model=LLM_PROVIDER_MODEL_STRING,
          api_key=API_KEY,
          temperature= 0.2,       # change the desired temperature
          max_tokens= 300,        # change the desired number of max tokens
          timeout=60,             # number of seconds before it retries. lengthen if your connection is spotty
          num_retries=3           # number of times it should retry after timing out
          )

      print(f"DSPy configured to use model: {LLM_PROVIDER_MODEL_STRING}")
  except Exception as e:
      print(f"ERROR: Could not configure DSPy with {LLM_PROVIDER_MODEL_STRING}. Exception: {e}")
      llm = None
else:
  print(f"ERROR: {API_KEY_NAME} not found. Please set it in Colab secrets OR as an environment variable.")

# Configure DSPy with the chosen LLM
if llm:
    dspy.settings.configure(lm=llm)
    print(f"DSPy global LM successfully configured. Active LM uses model: {llm.model if hasattr(llm, 'model') else LLM_PROVIDER_MODEL_STRING}")
else:
    print("CRITICAL ERROR: LLM was not configured. DSPy operations will fail. Please check your LLM_PROVIDER_MODEL_STRING and API key setup.")

GOOGLE_API_KEY loaded from Colab secrets.
DSPy configured to use model: gemini/gemma-3-1b-it
DSPy global LM successfully configured. Active LM uses model: gemini/gemma-3-1b-it


Next, we define the DSPy **Signature**, which acts as a blueprint for our LLM interactions. I've named it `AutomatabilityRatingSignature`. As you'll see in the code below, it provides a high-level description of the expected behavior. It also specifies:
* An **input field**: `full_request_text` (the complete, formatted prompt asking for the automatability rating).
* **Output fields**: A numeric `rating`.

Since the study's prompts vary on whether they ask for an explanation, we create a second signature `AutomatabilityRatingAndExplanationSignature`, identical to the first one, but including a second output field `explanation`.

This Signature isn't the prompt itself but rather a structured template. DSPy uses this, combined with the LLM configuration from the previous cell, to format requests and parse responses from any compatible model API.

To instruct DSPy to use this Signature for prompting the LLM, we wrap it in a `dspy.Predict` module, which we'll call `generate_rating`. At this point, we could already start sending requests to the model. For example:

```
# Example of calling generate_rating with a question about a skill:
# prediction = generate_rating(
#     full_request_text="Rate the automatability of the skill ~Writing~ in the context of advancements            \
#                        in generative AI in the next 5-10 years. Provide a rating from 1-5 and a brief           \
#                        explanation.")
# print(f"Rating: {prediction.rating}")
# print(f"Explanation: {prediction.explanation}")

# This might return something like:
#  Rating:'4',
#  Explanation:'Writing is a complex skill that can be automated to a significant degree by generative            \
#               AI for tasks like ...'
```
*(Note: The actual output structure is a `Prediction` object with attributes for each output field.)*

In [16]:
# Edit the signature if you want to modify the LLM's behaviour.
class AutomatabilityRatingSignature(dspy.Signature):
    """Given a specific request about a skill or work activity, provide a numerical rating."""

    full_request_text = dspy.InputField(
        desc="The complete, formatted prompt text.")

    rating = dspy.OutputField(
        desc="A single numerical rating on a scale of 1 (low) to 5 (high) (e.g., 1, 2, 3, 4, 5).")

class AutomatabilityRatingAndExplanationSignature(dspy.Signature):
    """Given a specific request about a skill or work activity, provide a numerical rating."""

    full_request_text = dspy.InputField(
        desc="The complete, formatted prompt text.")

    rating = dspy.OutputField(
        desc="A single numerical rating on a scale of 1 (low) to 5 (high) (e.g., 1, 2, 3, 4, 5).")

    explanation = dspy.OutputField(
        desc="A brief, concise explanation (1-3 sentences) for the rating.")

if dspy.settings.lm is None:
    print("CRITICAL ERROR: LLM not configured in dspy.settings. Please run the LLM configuration cell.")
else:
    generate_rating = dspy.Predict(AutomatabilityRatingSignature)
    generate_rating_explanation = dspy.Predict(AutomatabilityRatingAndExplanationSignature)
    print("Prediction modules initiliazed")

Prediction modules initiliazed


### Step 3 - Prompting the LLM

Now that everything is set up, we can move on to actually prompting the LLM, which turns out to be surprisingly simple.

In essence, for every item in our `items_to_rate` list (each representing a skill or work activity), the code iterates through all 10 `prompts` or phrasings. It inserts the item's name and description (when relevant) into the prompt template and then calls our `generate_rating()` function, sending the resulting text as the prompt to the LLM. The LLM's output (rating and explanation) is then saved, along with some identifying information, into a list of dictionaries called `all_results`.

In [ ]:
# Flag to signal breaking out of loops when the model can't be reached.
stop_processing_flag = False

# Empty list on which results will be saved
all_results = []

if 'generate_rating' not in locals():
    print("Error: The 'generate_rating' DSPy predictor is not initialized. Please check step 2.")
else:
    for item_info in items_to_rate:
        item_name = item_info.get('Name')
        item_descriptor = item_info.get('Description')
        # print(f"\nProcessing: '{item_name}'")     # uncomment for debugging or tracking progress

        for prompt_info in prompts:
            prompt_id = prompt_info.get('id')
            prompt_template = prompt_info.get('template')
            prompt_explanation = prompt_info.get('expects_explanation')

            if not prompt_template:
                print(f"Skipping prompt due to missing template: {prompt_info}")
                continue

            # Format the prompt - fill in the placeholders in the template
            try:
                if "{item_descriptor}" in prompt_template:
                    current_full_request_text = prompt_template.format(
                        item_name=item_name,
                        item_descriptor=item_descriptor
                    )
                else:
                    current_full_request_text = prompt_template.format(
                        item_name=item_name
                    )
            except KeyError as e:
                print(f"KeyError during formatting for item '{item_name}' with prompt ID '{prompt_id}'. Placeholder: {e}. Skipping.")
                continue

            # Call the DSPy Predictor
            try:
                # This sends the request to the LLM
                # Depending on whether the prompt calls for an explanation
                # we use generate_rating() or generate_rating_explanation()
                if prompt_explanation:
                  prediction = generate_rating_explanation(
                      full_request_text=current_full_request_text)
                else:
                  prediction = generate_rating(
                      full_request_text=current_full_request_text)

                # The prediction object will have attributes corresponding to the OutputFields
                llm_rating_raw = prediction.rating
                llm_explanation = prediction.explanation if hasattr(prediction, 'explanation') else ""

                result_entry = {
                    'item_name': item_name,
                    'prompt_id': prompt_id,
                    'expects_explanation_flag': prompt_explanation,
                    'llm_raw_rating_output': llm_rating_raw,
                    'llm_explanation_output': llm_explanation,
                }

                all_results.append(result_entry)

            except Exception as e:
                error_message = str(e)
                print(f"ERROR during LLM call for item '{item_name}' with prompt ID '{prompt_id}': {error_message}")

                # Store error information
                all_results.append({
                    'item_name': item_name,
                    'error': error_message
                })

                # Check if the error is model not found (404)
                if "404" in error_message:
                    print("This seems to be a critical API or model configuration error. Stopping further processing.")
                    stop_processing_flag = True # Signal to stop all processing
                    break

            # Rate Limiting - add a sleep timer to avoid hitting API rate limits.
            time.sleep(2.2) # Sleep for 2.2 seconds (adjust based on your model's rate limits)

        if stop_processing_flag:
            break # Critical error found, breaking out of both loops

    print("\n Processing Complete")
    print(f"Total results collected: {len(all_results)}")

### Step 4 - Store and process the results

The final step is to take the `all_results` list (which contains all the raw outputs from the LLM) and transform it into structured datasets that we can work with more easily. We'll use pandas for this, one of the most popular and powerful data manipulation libraries for Python. You can learn more about pandas [here](https://pandas.pydata.org/docs/).

We create two main tables (Pandas DataFrames):
1.  `full_table_df`: This table contains all 990 (99 items * 10 prompts) individual ratings, including the raw rating output and any explanation provided by the LLM for each specific prompt.
2.  `summary_table_df`: This table aggregates the responses from the different prompts for each unique skill or work activity. It calculates the mean score, standard deviation, and the range (minimum and maximum) of ratings. For further exploration, it also records the LLM's explanation corresponding to the lowest and highest recorded scores for which explanations were required.

In [ ]:
# Convert results to DataFrame
results_df = pd.DataFrame(all_results)
results_df["llm_raw_rating_output"] = pd.to_numeric(results_df["llm_raw_rating_output"],
                                                    errors = "coerce")

# Filter for desired columns
full_table_df = results_df[['item_name',
                         'prompt_id',
                         'expects_explanation_flag',
                         'llm_raw_rating_output',
                         'llm_explanation_output']].copy()

# Group by skill or work activity for aggregation
grouped_by_item = full_table_df.groupby('item_name')

# Calculate mean, std deviation, min and max
summary_stats = grouped_by_item['llm_raw_rating_output'].agg(
    average_rating = 'mean',
    std_dev = 'std',
    min = 'min',
    max = 'max',
).reset_index()

# Create the 'range' column
summary_stats['range'] = summary_stats.apply(
    lambda row: f"{int(row['min'])} - {int(row['max'])}"
    if pd.notna(row['min']) and pd.notna(row['max']) else "N/A",
    axis=1
)


def get_min_max_explanations(group):
    """
    Finds the min and max scores from the subset of prompts that expected an explanation,
    and returns those scores along with their corresponding explanations.
    """
    # Filter for prompts that expected an explanation
    explanation_group = group[group['expects_explanation_flag'] == True]

    if explanation_group.empty:
        return "No explanations were requested for this item."

    # Find the min and max ratings within filtered subset
    min_val = explanation_group['llm_raw_rating_output'].min()
    max_val = explanation_group['llm_raw_rating_output'].max()

    min_explanation_text = "N/A"
    max_explanation_text = "N/A"

    # Process the MINIMUM score's explanation
    if pd.notna(min_val):
        min_row = explanation_group[explanation_group['llm_raw_rating_output'] == min_val].iloc[0]
        explanation = min_row['llm_explanation_output']
        if isinstance(explanation, str) and explanation.strip():
            min_explanation_text = f"(Score: {int(min_val)}) {explanation.strip()}"

    # Process the MAXIMUM score's explanation
    if pd.notna(max_val):
        max_row = explanation_group[explanation_group['llm_raw_rating_output'] == max_val].iloc[0]
        explanation = max_row['llm_explanation_output']
        if isinstance(explanation, str) and explanation.strip():
            max_explanation_text = f"(Score: {int(max_val)}) {explanation.strip()}"

    # Format the final combined string
    if min_val == max_val and pd.notna(min_val):
        return min_explanation_text
    else:
        return f"Min: {min_explanation_text}\n----------\nMax: {max_explanation_text}"

min_max_explanations_series = grouped_by_item.apply(get_min_max_explanations, include_groups=False)
min_max_explanations_df = min_max_explanations_series.reset_index(name='min_max_explanations')

# Merge with summary_stats
summary_table_df = pd.merge(summary_stats, min_max_explanations_df, on='item_name')

print("Processed results into dataframes")

Finally, the two DataFrames (`full_table_df` and `summary_table_df`) are saved as CSV files. If you are running this in Google Colab, these files will appear in the session's file browser (usually accessible via a folder icon on the left sidebar), and you can download them from there for your records or further analysis.

In [ ]:
# Define the output directory
output_dir = "Results"

# Create the output directory if it doesn't already exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"Created directory: {output_dir}")

# Extract the model name and get current date
model_name_for_filename = LLM_PROVIDER_MODEL_STRING.split('/')[-1]
current_date = datetime.now().strftime('%Y-%m-%d')

# Create the full file paths, including the new directory
full_results_filepath = os.path.join(output_dir, f"full_results_{model_name_for_filename}_{current_date}.csv")
summary_filepath = os.path.join(output_dir, f"summary_results_{model_name_for_filename}_{current_date}.csv")

print(f"Saving full results to: {full_results_filepath}")
print(f"Saving summary to: {summary_filepath}")

# Save the resulting two tables into the specified directory
# If you're on Google Colab, make sure to download the files from the 'results'
# folder before restarting your session or you'll lose them.

full_table_df.to_csv(full_results_filepath, index=False, encoding='utf-8-sig')
summary_table_df.to_csv(summary_filepath, index=False, encoding='utf-8-sig')

print("\nFiles saved successfully")

### Next Steps

The results generated by this notebook should be viewed as an informative starting point rather than a definitive prediction. It is crucial to be critical of the outputs, especially when using lighter models for testing, as was done here for demonstration. While DSPy provides a more reliable prompting structure, choices regarding the Signature design, the specific LLM used, and its configuration parameters (temperature, max_tokens, etc.) can all influence the final ratings. More research is required to validate these methods and understand the optimal approaches for this type of assessment.

This project opens the door to many exciting avenues for further testing and research. For example:

* Impact of Phrasing: What is the impact of different prompt phrasings on the final ratings, and can this effect be predicted or controlled?
* Model Specialization: Could a base model be fine-tuned on literature about automation and skills for more accurate and consistent results?
* Context-Rich Ratings: Could the ratings be improved by providing the LLM with additional information, like specific job descriptions or recent labor market reports?
